# ELT Project

## Finding Data:

DATA SOURCE: https://www.kaggle.com/datasnaek/youtube-new/data <br/>
Utilizing: <br/>
3 csv files (Canada, US, and Mexico) <br/>
3 json file (Canada, US, and Mexico) <br/>

## Data Cleanup & Analysis

Plan and document the following:
* The sources of data that you will extract from.
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
* The type of final production database to load the data into (relational or non-relational).
* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report:

Submit a Final Report that describes the following:
* Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* Transform: what data cleaning or transformation was required.
* Load: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

In [1]:
#dependencies 
import os
import pandas as pd
import json
from sqlalchemy import create_engine
from bs4 import BeautifulSoup 

In [2]:
#reading in Canada csv video info
# ca_file= "data/CAvideos.csv"
# CA_df =pd.read_csv(ca_file)
# CA_df.head()

In [3]:
# #adding a column to define which country info came from
# CA_df.insert(1,"country", "CA") 
# CA_df.head()

In [4]:
#reading in Great Britain csv video info
# gb_file= "data/GBvideos.csv"
# GB_df =pd.read_csv(gb_file)
# GB_df.head()

In [5]:
# #adding a column to define which country info came from
# GB_df.insert(1,"country", "GB")
# GB_df.head()

In [6]:
#reading in United States csv video info
# us_file= "data/USvideos.csv"
# US_df =pd.read_csv(us_file)
# US_df.head()

In [7]:
# #adding a column to define which country info came from
# US_df.insert(1,"country", "US")
# US_df.head()

In [15]:
#merging Canada and Great Britain
merge = CA_df.merge(GB_df, on='video_id')
merge.head()

,video_id,country_x,trending_date_x,title_x,channel_title_x,category_id_x,publish_time_x,tags_x,views_x,likes_x,...,tags_y,views_y,likes_y,dislikes_y,comment_count_y,thumbnail_link_y,comments_disabled_y,ratings_disabled_y,video_error_or_removed_y,description_y
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",20539417,840642,47715,124236,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",22702386,869304,50018,123227,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",24578152,891283,51977,125444,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",26448434,911883,53873,127481,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...


In [16]:
#mering us and other two countries
countries_merged = merge.merge(US_df, on ='video_id')
countries_merged.head()

,video_id,country_x,trending_date_x,title_x,channel_title_x,category_id_x,publish_time_x,tags_x,views_x,likes_x,...,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158531,787419,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",20539417,840642,47715,124236,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",22702386,869304,50018,123227,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",24578152,891282,51977,125444,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",26448434,911883,53873,127481,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...


In [8]:
# #viewing all merged columns to weed out the ones needing to drop
# countries_merged.columns

In [9]:
# #drop unnecessary columns not using for project
# countries_merged.drop(['tags_x', 'comment_count_x', 'thumbnail_link_x', 'comments_disabled_x', 'ratings_disabled_x','video_error_or_removed_x', 'tags_y', 'comment_count_y', 'thumbnail_link_y', 'comments_disabled_y', 'ratings_disabled_y', 'video_error_or_removed_y',  'tags', 'comment_count',
#        'thumbnail_link', 'comments_disabled', 'ratings_disabled','video_error_or_removed','publish_time_x','publish_time_y','publish_time'] , axis=1, inplace=True)
# countries_merged.head()

In [10]:
# #seeing all coulmns in left merged df
# countries_merged.columns

In [11]:
# #renaming columns by countries
# #these are the coloumns to be renamed
# # 'video_id', 
# # CA= 'country_x', 'trending_date_x', 'title_x','channel_title_x', 'category_id_x','views_x', 'likes_x', 'dislikes_x', 'description_x', 
# # GB = 'country_y','trending_date_y', 'title_y', 'channel_title_y', 'category_id_y', 'description_y','views_y', 'likes_y', 'dislikes_y',
# # US = 'country', 'trending_date', 'title', 'channel_title', 'category_id','views', 'likes', 'dislikes','description'

# countries_df=countries_merged.rename(columns={
#     'country_x':'Canada','trending_date_x':'trend_date_CA','title_x':'title_CA','channel_title_x':'channel_title_CA','category_id_x':'category_id_CA','views_x':'views_CA', 'likes_x':'likes_CA', 'dislikes_x': 'dislikes_CA', 'description_x':'discriptions_CA',
#     'country_y':'Great Britain','trending_date_y':'trend_date_GB', 'title_y':'title_GB', 'channel_title_y':'channel_title_GB', 'category_id_y':'category_id_GB','views_y':'views_GB','likes_y':'likes_GB', 'dislikes_y':'dislikes_GB','description_y':'discriptions_GB',
#     'country':'United States', 'trending_date': 'tend_date_US', 'title':'title_US', 'channel_title':'channel_title_US', 'category_id':'category_id_US','views':'views_US', 'likes':'likes_US', 'dislikes':'dislikes_US','description':'description_US'})
# countries_df.head()